# Parkinson's Disease Progression Prediction: Linear and Isotonic Groups

This notebook shows a model which is based on two assumptions:
1. We can identify the healthy control group because it is examined only once a year.
2. Symptoms get worse with age, even in the control group.

The model does not look at the proteins.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import os, gc, pickle, datetime, lightgbm, math, catboost, xgboost, warnings, scipy.optimize
from colorama import Fore, Back, Style
import amp_pd_peptide

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import check_consistent_length
from sklearn.model_selection import KFold, GroupKFold, cross_val_score
from sklearn.dummy import DummyRegressor
from sklearn.metrics import make_scorer

np.set_printoptions(linewidth=150, edgeitems=5)

# Reading the data

In [ ]:
# trpep = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv',
#                     index_col='visit_id')
# with pd.option_context("display.min_rows", 4):
#     display(trpep)

trpro = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv',
                    index_col='visit_id')
print('trpro')
with pd.option_context("display.min_rows", 4):
    display(trpro)

trcli = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv',
                    index_col='visit_id')
print('trcli')
with pd.option_context("display.min_rows", 4):
    display(trcli)

Creating mock input and API for testing...

In [ ]:
from typing import Sequence, Tuple
class MockApi:
    def __init__(self):
        '''
        YOU MUST UPDATE THE FIRST THREE LINES of this method.
        They've been intentionally left in an invalid state.

        Variables to set:
            input_paths: a list of two or more paths to the csv files to be served
            group_id_column: the column that identifies which groups of rows the API should serve.
                A call to iter_test serves all rows of all dataframes with the current group ID value.
            export_group_id_column: if true, the dataframes iter_test serves will include the group_id_column values.
        '''
        self.input_paths: Sequence[str] = ['mock_train.csv',
                                           '/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv',
                                           '/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv',
                                           'mock_sample_submission.csv'
                                          ]
        self.group_id_column: str = 'visit_month'
        self.export_group_id_column: bool = True
        # iter_test is only designed to support at least two dataframes, such as test and sample_submission
        assert len(self.input_paths) >= 2

        self._status = 'initialized'
        self.predictions = []

    def iter_test(self) -> Tuple[pd.DataFrame]:
        '''
        Loads all of the dataframes specified in self.input_paths,
        then yields all rows in those dataframes that equal the current self.group_id_column value.
        '''
        if self._status != 'initialized':
            raise Exception('WARNING: the real API can only iterate over `iter_test()` once.')

        dataframes = []
        for pth in self.input_paths:
            dataframes.append(pd.read_csv(pth, low_memory=False))
        group_order = dataframes[0][self.group_id_column].drop_duplicates().tolist()
        dataframes = [df.set_index(self.group_id_column) for df in dataframes]

        for group_id in group_order:
            print('group_id:', group_id)
            self._status = 'prediction_needed'
            current_data = []
            for df in dataframes:
                try:
                    cur_df = df.loc[group_id].copy()
                    # returning single line dataframes from df.loc requires special handling
                    if not isinstance(cur_df, pd.DataFrame):
                        cur_df = pd.DataFrame({a: b for a, b in zip(cur_df.index.values, cur_df.values)}, index=[group_id])
                        cur_df = cur_df.index.rename(self.group_id_column)
                except KeyError:
                    cur_df = df.loc[[]].copy()
                cur_df = cur_df.reset_index(drop=not(self.export_group_id_column))
                current_data.append(cur_df)
            yield tuple(current_data)

            while self._status != 'prediction_received':
                print('You must call `predict()` successfully before you can continue with `iter_test()`', flush=True)
                yield None

        with open('mock_submission.csv', 'w') as f_open:
            pd.concat(self.predictions).to_csv(f_open, index=False)
        self._status = 'finished'

    def predict(self, user_predictions: pd.DataFrame):
        '''
        Accepts and stores the user's predictions and unlocks iter_test once that is done
        '''
        if self._status == 'finished':
            raise Exception('You have already made predictions for the full test set.')
        if self._status != 'prediction_needed':
            raise Exception('You must get the next test sample from `iter_test()` first.')
        if not isinstance(user_predictions, pd.DataFrame):
            raise Exception('You must provide a DataFrame.')

        self.predictions.append(user_predictions)
        self._status = 'prediction_received'

# Create mock_train.csv
temp_list = []
for i in range(1, 5):
    temp = trcli[['patient_id', 'visit_month']].copy()
    temp['level_3'] = i
    temp['updrs_test'] = f'updrs_{i}'
    temp_list.append(temp)
mock_train = pd.concat(temp_list)
mock_train.to_csv('mock_train.csv')

# Create mock_sample_submission.csv
temp_list = []
for wait in [0, 6, 12, 24]:
    temp = mock_train.copy()
    temp['wait'] = wait
    temp_list.append(temp)
y = pd.concat(temp_list)
y = y[y.visit_month + y.wait <= 108]
y['prediction_id'] = (y[['patient_id', 'visit_month', 'wait', 'level_3']]
                      .apply((lambda r: f"{r.patient_id}_{int(r.visit_month)}_updrs_{r.level_3}_plus_{r.wait}_months"), axis=1))
y['rating'] = 0
y[['prediction_id', 'rating', 'visit_month']].to_csv('mock_sample_submission.csv')
del temp_list, mock_train, y

# The metric

We define the metric as a function and as a scikit-learn scorer object.

In [ ]:
# Define the metric
def smapep1(y_true, y_pred):
    """SMAPE of y+1, a nonnegative float, smaller is better
    
    Parameters: y_true, y_pred: array-like
    
    Returns 100 for 100 % error.
    y_true may have missing values.
    """
    check_consistent_length(y_true, y_pred)
    y_true = np.array(y_true, copy=False).ravel()
    y_pred = np.array(y_pred, copy=False).ravel()
    y_true, y_pred = y_true[np.isfinite(y_true)], y_pred[np.isfinite(y_true)]
    if (y_true < 0).any(): raise ValueError('y_true < 0')
    if (y_pred < 0).any(): raise ValueError('y_pred < 0')
    denominator = (y_true + y_pred) / 2 + 1
#     print(np.abs(y_pred - y_true), denominator)
    ape = np.abs(y_pred - y_true) / denominator
    return np.average(ape) * 100

# The scorer returns nonpositive values so that greater is better.
# It will be used as an argument to cross_val_score
smapep1_scorer = make_scorer(smapep1, greater_is_better=False)


We define a function which returns the best constant prediction for a given list of y values.

In [ ]:
# SMAPE Optimizer
def optimize_smapep1(y_true):
    """Returns the single prediction which optimizes smapep1 for all y_true
    
    Parameter: y_true: 1d array-like
    
    Nan values in y are ignored.
    The implementation is inefficient for large max(y) - min(y)"""
    y = np.array(y_true)
    if y.ndim != 1: raise ValueError('y must be one-dimensional')
    y = y[np.isfinite(y)]
    best_i, best_smape = None, 5000
    for i in range(int(min(y)), int(max(y))+1):
        s = smapep1(y, np.full(len(y), i))
        if s < best_smape:
            best_i, best_smape = i, s
    return best_i


# Analysis of the three populations

If we plot the median updrs scores for every month, we see that the months which are multiples of 12 (the cyan markers on the gridlines) are usually lower than the non-multiples of 12 (the magenta markers between the gridlines). This cannot be a coincidence.

In [ ]:
# Plot the median updrs for every visit_month
base = trcli
visit_months = base.visit_month.unique()
plt.figure(figsize=(10, 6))
plt.suptitle('Time series median of patients')
for i in range(1, 5):
    plt.subplot(2, 2, i)
    plt.title(f'updrs_{i}')
    temp = base.groupby('visit_month').median()
    plt.plot(temp.index, temp[f'updrs_{i}'], label='median', lw=3, color='k')
    plt.scatter(temp.index, temp[f'updrs_{i}'], c=temp.index % 12 != 0, cmap='cool', s=100)
    plt.legend()
    plt.xticks(np.linspace(0, 108, 10))
    plt.grid(axis='x')
    plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
    plt.ylabel('updrs')
    if i >= 3: plt.xlabel('visit_month')
plt.tight_layout()
plt.show()

A scatterplot of the 248 patients versus the months of their updrs assessments reveals that there are three groups of patients:
- The patients of the green group had their first visits in months 0, 3, 6, 9, 12.
- The patients of the orange group had their first visits in months 0, 6, 12, 18, 24 and the last visit in month 60.
- The patients of the red group had their first visits in months 0, 12, 24.

In [ ]:
# Copy the trcli dataframe and renumber the patients from 0 through 247
train = trcli.copy()
le = LabelEncoder()
train['patient_enc'] = le.fit_transform(train['patient_id'])

# Define the three disjoint populations: m3, m6 and rest
# (sets of patient_enc)
m3 = set(train[train.visit_month % 6 == 3].patient_enc)
m6 = set(train[train.visit_month % 12 == 6].patient_enc)
m6 = m6.difference(m3)

# Define the order of the patients in the diagram
# (array with one entry per patient)
patient_sort = np.full(len(train['patient_id'].unique()), 99999)
patient_sort[list(m3)] = np.arange(len(m3))
patient_sort[list(m6)] = np.arange(len(m3), len(m3)+len(m6))
patient_sort[patient_sort == 99999] = np.arange(len(m3)+len(m6), len(patient_sort))
train['patient_sort'] = patient_sort[train['patient_enc']] # x coordinate of dataframe row in diagram

# Define the color of the patients
patient_col = np.full(len(train['patient_id'].unique()), 'r', dtype=object)
patient_col[list(m3)] = 'g'
patient_col[list(m6)] = 'orange'
train['patient_col'] = patient_col[train['patient_enc']]

plt.figure(figsize=(13, 4))
plt.suptitle('All clinical visits of all patients')
plt.scatter(train['patient_sort'], train['visit_month'], s=5, c=train['patient_col'])
plt.xlabel('patient')
plt.ylabel('month')
plt.yticks(train['visit_month'].unique())
plt.show()

trpro['patient_enc'] = le.transform(trpro['patient_id'])
trpro['patient_sort'] = patient_sort[trpro['patient_enc']]
trpro['patient_col'] = patient_col[trpro['patient_enc']]
temp = trpro[['patient_sort', 'visit_month', 'patient_col']].drop_duplicates()

plt.figure(figsize=(13, 4))
plt.suptitle('All protein measurements of all patients')
plt.scatter(temp['patient_sort'], temp['visit_month'], s=5, c=temp['patient_col'])
plt.xlabel('patient')
plt.ylabel('month')
plt.yticks(temp['visit_month'].unique())
plt.show()


If we plot the updrs scores over time of every patient, we see differences among the groups. The red group in particular has the lowest updrs scores, which means that these are the healthiest people, and updrs_4 has rarely been measured for them.

We can hypothesize that the red group is the control group (a group of people without Parkinson's disease), and the experimenters decided to test the control group only once a year and to skip the updrs_4 test for this group. The real patients (green and orange groups) were tested more often and with all four updrs tests.

In [ ]:
visit_months = train.visit_month.unique()
plt.figure(figsize=(10, 6))
plt.suptitle('Time series per patient')
for i in range(1, 5):
    plt.subplot(2, 2, i)
    plt.title(f'updrs_{i}')
    for patient_enc in sorted(set(train.patient_enc)):
        pat_df = train[train.patient_enc == patient_enc]
        plt.plot(pat_df.visit_month, pat_df[f'updrs_{i}'], lw=1, color=patient_col[patient_enc], alpha=0.5)
    plt.xticks(visit_months)
    plt.ylabel('updrs')
    if i >= 3: plt.xlabel('visit_month')
plt.tight_layout()
plt.show()

If we plot the medians of the three time series, the differences between the three groups become much more visible:

In [ ]:
# Plot the median updrs for every visit_month by population group
base = train
visit_months = base.visit_month.unique()
plt.figure(figsize=(10, 6))
plt.suptitle('Time series median of patients per group')
for i in range(1, 5):
    plt.subplot(2, 2, i)
    for col, label in zip(['g', 'orange', 'r'], ['m3', 'm6', 'm12 (healthy)']):
        temp2 = base[base.patient_col == col].groupby('visit_month').median()
        plt.plot(temp2.index, temp2[f'updrs_{i}'], color=col, alpha=0.5, lw=3, label=label)
    plt.legend()
    plt.xticks(np.linspace(0, 108, 10))
    plt.ylabel(f'updrs_{i}')
    plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
    plt.xlabel('visit_month')
plt.tight_layout()
plt.show()

Conclusion: We can distinguish the control group from the real patients according to their first non-zero `visit_month`: If the first non-zero `visit_month` is <12, we have a real patient; if the first non-zero `visit_month` equals 12, the person belongs to the healthy control group. This distinction has high predictive value for the updrs scores.

# Training data preprocessing

In [ ]:
month_array = trcli.visit_month.unique() # [0, 3, 6, ..., 108]
patient_array = trcli.patient_id.unique()

# Month of first protein measurement: 0 for most patients, 3, 6, 12, 24 for a few
first_month = trpro.groupby('patient_id').visit_month.min()

def update_group(group, visit_month):
    """Update the group assignment of a patient"""
    if visit_month % 12 != 0 or group == 'ill': return 'ill'
    if visit_month >= 12: return 'healthy'
    return 'unknown'

# We create a dataframe df which contains one row per
# (patient_id, visit_month, pred_month) triple of the training data
tr_list = []
for patient_id in patient_array:
    group = 'unknown'
    for visit_month in month_array:
        if ((trcli.patient_id == patient_id) & (trcli.visit_month == visit_month)).sum() == 0: continue
        group = update_group(group, visit_month)
        if first_month.loc[patient_id] <= visit_month and visit_month <= 84:
            for wait in [0, 6, 12, 24]:
                pred_month = visit_month + wait
                tr_list.append((patient_id, visit_month, pred_month, group))
df = pd.DataFrame(tr_list, columns=['patient_id', 'visit_month', 'pred_month', 'group'])
df = df.merge(trcli, left_on=['patient_id', 'pred_month'], right_on=['patient_id', 'visit_month'])
print('Training data shape:', df.shape) # (7646, 10)
print(df['group'].value_counts())
print()

# The model

We can now define our model (which does not distinguish between green and orange):
- In month 0, we can not yet distinguish the groups, so we predict a constant which is optimized to give the best smapep1 score over all patients.
- Starting at month 3, we can distinguish two groups (red vs. non-red), and we predict different updrs scores depending on the group.

As there is no cure for Parkinson's disease and the patients get older during the course of the experiment, we assume that the symptoms must get worse with age. We fit a linear model for the patients, and for the control group we fit an [isotonic regression model](https://en.wikipedia.org/wiki/Isotonic_regression) to predict the monotonically increasing updrs scores from `visit_month`. As the [isotonic regression of scikit-learn](https://scikit-learn.org/stable/modules/isotonic.html) optimizes mse and this competition is scored by smape, we need to implement our own method.

In [ ]:
def calculate_predictions(pred_month, trend, target):
    if target == 'updrs_4': pred_month = pred_month.clip(60, None)
    return np.round(trend[0] + pred_month * trend[1]).clip(0, None)

def optimize_smapep1_linear(pred_month, y, target):
    """Find the optimum linear regression model for one feature and one target
    
    This function optimizes smapep1.
    """
    def function_to_minimize(x):    
        metric = smapep1(
            y_true=y, 
            y_pred=calculate_predictions(
                pred_month=pred_month,
                trend=x,
                target=target
            )
        )
        return metric

    # As the function to minimize is not convex, we give the optimizer some help
    bounds = {'updrs_1': [(3, 7), (0, 0.1)],
              'updrs_2': [(3, 7), (0, 0.1)],
              'updrs_3': [(16, 22), (0, 0.3)],
              'updrs_4': [(-7, -5), (0, 0.2)],
             }
    trend = scipy.optimize.minimize(
        fun=function_to_minimize,
        x0=[bounds[target][0][0]+1, bounds[target][1][0]+0.05],
        bounds=bounds[target],
        method='Powell'
    ).x
    # print(target, trend.round(3))
    
    return list(trend)

def optimize_smapep1_isotonic(pred_month, y):
    """Find the optimum monotonic regression model for one feature and one target
    
    This function optimizes smapep1.
    """
    pred_month = np.array(pred_month).astype(int)
    y = np.array(y)
    if pred_month.ndim != 1: raise ValueError('dimension of pred_month must be 1')
    if y.ndim != 1: raise ValueError('dimension of y must be 1')
        
    unique_months = np.unique(pred_month)
    encoding = np.full(unique_months.max()+1, -1)
    encoding[unique_months] = np.arange(len(unique_months))
    pred_month_encoded = encoding[pred_month]
    
    y_hat = np.zeros(len(unique_months))
    while True:
        i = len(unique_months) - 1
        best_score, best_i = smapep1(y, y_hat[pred_month_encoded]), None
        while i >= 0:
            new_y_hat = y_hat.copy()
            new_y_hat[i:] += 1
            score = smapep1(y, new_y_hat[pred_month_encoded])
            if score < best_score:
                best_score, best_i = score, i
            i -= 1
        if best_i is None: break
        y_hat[best_i:] += 1
        
    return unique_months, encoding, y_hat

class IsotonicGroups(BaseEstimator, RegressorMixin):
    """Model updrs scores as a function of the features 'group' and 'pred_month'
    """

    def fit(self, X, y):
        """
        Fit the estimator.
        
        X: pd.DataFrame with columns 'group' and 'pred_month'
           'group' has the three values 'unknown', 'healthy' and 'ill'
           'pred_month' has integer values in the range 0..108
        y: pd.DataFrame with 1 or 4 columns 'updrs_?'
        """
        if type(X) is not pd.DataFrame: raise TypeError('X must be DataFrame')
        if type(y) is not pd.DataFrame: raise TypeError('y must be DataFrame')
        self.unknown_ = y[X['group'] == 'unknown'].apply(lambda s: optimize_smapep1(s))
        
        self.trends_ = {}
        for group in ['healthy', 'ill']:
            pred_month = X[X['group'] == group]['pred_month'] # 1d
            y_group = y[(X['group'] == group).values] # 2d
            if group == 'healthy':
                # Optimize every column
                optimized = y_group.apply(lambda s: optimize_smapep1_isotonic(pred_month, s))
                # Optimized is a dataframe with 3 rows and four columns
                unique_months = optimized.iloc[0, 0] # all four columns of row 0 are identical
                encoding = optimized.iloc[1, 0] # all four columns of row 1 are identical
                y_hat = np.column_stack(list(optimized.iloc[2]))
                self.trends_[group] = unique_months, encoding, y_hat
            else: # 'ill'
                unique_months = np.unique(pred_month)
                encoding = np.full(unique_months.max()+1, -1)
                encoding[unique_months] = np.arange(len(unique_months))
                trend_list = []
                for target in y_group.columns: # [f'updrs_{i}' for i in range(1, 5)]:
                    trend = optimize_smapep1_linear(pred_month, y_group[target], target)
                    trend_list.append(calculate_predictions(unique_months, trend, target))
                y_hat = np.column_stack(trend_list)
                self.trends_[group] = unique_months, encoding, y_hat
        return self
    
    def predict(self, X):
        v = np.zeros((len(X), len(self.unknown_)))
        v[X['group'] == 'unknown'] = self.unknown_.values.reshape(1, -1)
        for group in ['healthy', 'ill']:
            pred_month = X[X['group'] == group]['pred_month'].values
            _, encoding, y_hat = self.trends_[group]
            pred_month_encoded = encoding[pred_month]
            v[X['group'] == group] = y_hat[pred_month_encoded]
        return v
    
    def display_coefficients(self):
        """Print all parameters of the model and plot the predictions"""
        print('Unknown:')
        print(self.unknown_.to_frame().reset_index(drop=True))
        print()
        for group in ['ill', 'healthy']:
            print(f'{group}:')
            unique_month, encoding, y_hat = self.trends_[group]
            with pd.option_context("display.width", 160):
                print(pd.DataFrame(y_hat.astype(int), index=unique_month).T)
            print()
                
        # Plot the predictions
        plt.figure(figsize=(10, 6))
        plt.suptitle('Model predictions')
        for i in range(1, 1+len(self.unknown_)):
            plt.subplot(2, 2, i)
            plt.scatter([0], self.unknown_.iloc[i-1], color='saddlebrown', s=50, label='unknown')
            for group, col in zip(['ill', 'healthy'], ['greenyellow', 'r']):
                unique_month, encoding, y_hat = self.trends_[group]
                plt.scatter(unique_month, y_hat[:, i-1], color=col, alpha=0.5, s=50, label=group)
                # plt.plot(unique_month, y_hat[:, i-1], color=col, alpha=0.5, lw=3, label=group)
            plt.legend()
            plt.xticks(np.linspace(0, 108, 10))
            plt.ylabel(f'updrs_{i}')
            plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
            plt.xlabel('visit_month')
        plt.tight_layout()
        plt.show()
    

# Cross-validation


In [ ]:
print('Cross-validation scores')
overall_score = []
for i in range(1, 5):
    cvs = cross_val_score(IsotonicGroups(),
                          X=df, y=df[[f'updrs_{i}']],
                          groups=df['patient_id'],
                          scoring=smapep1_scorer,
                          cv=GroupKFold(n_splits=10),
                          error_score='raise')
    print([f'updrs_{i}:'], -cvs.round(1), -cvs.mean().round(1))
    overall_score.append(-cvs)
print(f'Overall cv score of the group model: {np.array(overall_score).mean():.2f}')
# Score = 53.68 (healthy isotonic, ill linear, unknown constant)

# Training

We retrain the model on the full training set and display its parameters:

In [ ]:
model = IsotonicGroups()
model.fit(df, df[[f'updrs_{i}' for i in range(1, 5)]])
model.display_coefficients()


# Submission

In [ ]:
def get_predictions(test, sample_submission, model):
    """Return the submission dataframe for the given test data and fitted model"""
    
    # Determine groups
    for _, row in test[['patient_id', 'visit_month']].drop_duplicates().iterrows():
        patient_id, visit_month = row.patient_id, row.visit_month
        group = group_dict.get(patient_id, 'm00')
        group = update_group(group, visit_month)
        group_dict[patient_id] = group
        print(f"{patient_id:6} {visit_month:3} {group}")
            
    # Construct X
    # prediction_id example: '3342_6_updrs_1_plus_12_months'
    X = pd.DataFrame(list(sample_submission.prediction_id.str.split('_')),
                     columns=['patient_id', 'visit_month', 'updrs', 'updrs_num', 'plus', 'wait', 'months'])
    X = X[['patient_id', 'visit_month', 'updrs_num', 'wait']].astype(int)
    X['pred_month'] = X['visit_month'] + X['wait']
    X['group'] = X.patient_id.map(group_dict)
    
    # Predict y and reformat
    y = model.predict(X) # 2d array of shape (n_samples, 4)
    sample_submission['rating'] = y[np.arange(len(y)), X['updrs_num'] - 1] # select 1 per row
    sample_submission['rating'] = sample_submission['rating'].fillna(0)
    return sample_submission

In [ ]:
env = amp_pd_peptide.make_env()
# env = MockApi()
group_dict = {}
iter_test = env.iter_test()  
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    result = get_predictions(test, sample_submission, model)
    display(result)
    env.predict(result)